# Efficient Large-Scale Language Model Training on GPU Clusters Using Megatron-LM

**Focus**: 3. Performance Analysis of Parallelization Configurations

**References**: 
- Efficient Large-Scale Language Model Training on GPU Clusters
Using Megatron-LM: https://arxiv.org/pdf/2104.04473
- Nvidia/Docs/Using NCCL (Nvidia Collective Communications Library)/Operations: https://docs.nvidia.com/deeplearning/nccl/user-guide/docs/usage/operations.html#reducescatter

**Purpose**: 

**Approach**: 

**Result**: 


**Definitions**: 

**Notes**:

Pre-requisites to complete the communication analysis
NCCL (Nvidia Collective Communications Library) Operations:

1. All-Reduce: reductions on data (ex. sum) across devices and writes the result int he receive buffers of *every* rank

2. Broadcast: copies N-element buffer on the root rank to all ranks

3. Reduce: same operation as All-Reduce, but writes the result in the receive buffer of the specified root rank
- All-Reduce = Reduce -> Broadcast

4. All-Gather: each K processors aggregates N (sharded) values for every processor into an output dimension K*N
- *Ring* All-Reduce (bandwidth optimal) = Reduce Scatter -> All Gather, executed over a logical topological ring

5. Reduce-Scatter: Reduce, but output is sharded col-wise across ranks.


3.2 Tensor and Pipeline Model Parallelism
- The key idea is that TP communication cost per microbatch = l^{stage} * (8bsh((t-1)/t))
    - Derivation:
        - Ring All-Reduce = Reduce Scatter + All-Gather
        - Reduce Scatter = t-1 chunks are sent out and received per GPU, where chunk size = (N/t). Therefore, the data communication cost per GPU = (t-1) * (N/t)
        - All-Gather = same data cost as above
        - Ring All-Reduce Cost = 2 * (t-1) * (N/t)
        - Forward Pass = 1 Attn + 1 FF Ring All-Reduce Operations = 4 * (t-1) * (N/t)
        - Forward + Backward Pass = 8 * (t-1) * (N/t) = 8N*((t-1)/t) per device per layer
- Therefore, the takeaway is that we should apply TP size = all GPUs in a server/node, and then PP size across nodes to reduce the communication cost.

**FAQs**:

**Action items**: